In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'}}

In [2]:
model_name = "ensemble_baseline"

In [3]:
import pandas as pd
import numpy as np

np.random.seed(seed=0)

In [4]:
train_data = pd.read_pickle(settings["train"]["pickle_path"])
valid_data = pd.read_pickle(settings["valid"]["pickle_path"])

train_data = pd.concat([train_data,valid_data],axis = 0)

test_data = pd.read_pickle(settings["test"]["pickle_path"])

In [5]:
columns = ['A_dist_bert', 'B_dist_bert',
       'A_pos_bert', 'B_pos_bert', 'pron_pos_bert', 'topic_A', 'topic_B', 
       'A_vector_bert_256_mean', 'B_vector_bert_256_mean',
       'pron_vector_bert_256_mean', 'product_vector_A_bert_256',
       'product_vector_B_bert_256']

In [6]:
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = np.array(train_data.label)


X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)
y_test = np.array(test_data.label)

In [8]:
from joblib import dump, load
model_name_list = ["LR_256_baseline","SVM_256_baseline"]

for model in model_name_list:
    print (model)
    tr,ts = load(model +'+basic.joblib')
    X_train = np.concatenate([X_train,tr],axis = 1)

    X_test = np.concatenate([X_test,ts],axis = 1)

LR_256_baseline
SVM_256_baseline


In [36]:
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

score = []
kf = KFold(n_splits=5) 
for train_index, test_index in kf.split(X_train):
    X_tr, X_ts = X_train[train_index], X_train[test_index]
    y_tr, y_ts = y_train[train_index], y_train[test_index]
    lr = LogisticRegression(penalty = "l1",solver = "saga",random_state=0,multi_class='multinomial', C= 0.17).fit(X_tr, y_tr)
    pred_ts = lr.predict_proba(X_ts)
    s = log_loss(y_ts,pred_ts)
    score.append(s)
    

    

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: C

In [37]:
score

[0.5518518046726659,
 0.5687163993344871,
 0.5460859518414047,
 0.4788524885115653,
 0.5190967497755528]

In [38]:
np.array(score).mean()

0.5329206788271351

In [40]:
lr = LogisticRegression(penalty = "l1",solver = "saga",random_state=0,multi_class='multinomial', C= 0.17).fit(X_train, y_train)
pred_lr= lr.predict_proba(X_test)

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [41]:
log_loss(y_test,pred_lr)

0.3340959758829121

In [42]:
sub_df = pd.read_csv("../test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_lr
sub_df.to_csv("../test_and_submit/submission+model+"+model_name+"@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.896888,0.052602,0.050510
1,development-2,0.916337,0.063210,0.020454
2,development-3,0.093635,0.846779,0.059586
3,development-4,0.041777,0.713620,0.244603
4,development-5,0.004922,0.987575,0.007503


In [43]:
X_train = np.concatenate([X_train,X_test],axis = 0)
y_train = np.concatenate([y_train,y_test],axis = 0)

lr = LogisticRegression(penalty = "l1",solver = "saga",random_state=0,multi_class='multinomial', C= 0.038).fit(X_train, y_train)

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [44]:
from joblib import dump, load
dump(lr, model_name +'.joblib') 

['ensemble_baseline.joblib']